In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from os import listdir
from os.path import isfile, join


from src.SimilarityArtworks import *
from src.SimilarityUsers import *
from src.ArtworksMatrix import *
from src.UsersMatrix import *
from src.UsersClustering import *
from src.AverageUser import *

In [2]:
import inspect

def inner_classes_list(cls):
    return [cls_attribute.__name__ for cls_attribute in cls.__dict__.values()
            if inspect.isclass(cls_attribute)]
            # and issubclass(cls_attribute, SimilarityFunctionInterface)]
    
    
import functools

def on_button_clicked(b, rs_="some_default_string"):
    fun(rs_)

# button.on_clicked(functools.partial(on_button_clicked, rs_="abcdefg"))

In [3]:
from traitlets import traitlets

class LoadedButton(widgets.Button):
    """A button that can holds a value as a attribute."""

    def __init__(self, value=None, *args, **kwargs):
        super(LoadedButton, self).__init__(*args, **kwargs)
        # Create the value attribute.
        self.add_traits(value=traitlets.Any(value))

def add_num(ex):
    ex.value = ex.value+1
    print(ex.value)

lb = LoadedButton(description="Loaded", value=1)
lb.on_click(add_num)
display(lb)

LoadedButton(description='Loaded', style=ButtonStyle())

In [26]:
class foo(object):

    def __init__(self, path='data/'):
      self.path = path
      self.files = [f for f in listdir(self.path) if isfile(join(self.path, f))]
    
    
    ### 1. CARGAR DATOS ###
    def loadData(self):
      self.users = pd.read_csv(self.path + self.usersDropdown.value, index_col=0)
      self.items = pd.read_csv(self.path + self.itemsDropdown.value)
      self.users['positive'] = self.users['positive'].apply(eval)
      self.users['negative'] = self.users['negative'].apply(eval)
      self.users['mixed'] = self.users['mixed'].apply(eval)
      self.users_sim_functions = [None]
      self.users_sim_functions.extend(inner_classes_list(SimilarityUsers))
      self.items_sim_functions = [None]
      self.items_sim_functions.extend(inner_classes_list(SimilarityArtworks))
    
    def loadData_create_widgets(self):
      self.loadDataComplete = widgets.Label(value="- Users and items data loaded -")
      self.loadDataTitle = widgets.HTML(value="<b>Select data files</b>")
      self.usersDropdown = widgets.Dropdown(options=self.files,
                                            value='Prado_users_emotions_OnePolarity.csv',
                                            description='Users:',
                                            disabled=False,
                                            )
      self.itemsDropdown = widgets.Dropdown(options=self.files,
                                            value='Prado_artworks_wikidata.csv',
                                            description='Items:',
                                            disabled=False,
                                            )
      self.loadDataButton = widgets.Button(description="Done")
      self.loadDataButton.on_click(self.on_loadDataButton_clicked)

    def on_loadDataButton_clicked(self, change):
      self.out.clear_output()
      self.loadData()
      self.addItemsAtribute_create_widgets()
      self.atribute_list = []
      with self.out:
        display(self.loadDataComplete)
        display(self.addItemsAtributeTitle)
        display(self.addItemsAtributeButton)
    
    ### 2. SIMILITUD ITEMS ###
    def computeItemsSim(self):
      funcs = {x : None for x in self.items.columns}
      weights = {x : None for x in self.items.columns}  

      for atr in self.atribute_list:
        funcs.update({atr.children[0].value : atr.children[1].value})
        weights.update({atr.children[0].value : atr.children[2].value})

      AM = ArtworksMatrix(self.items, function_dict=funcs, weight_dict=weights)
      self.itemsMatrix = AM.getSimilarityMatrix()
      self.itemsMatrix
    
    def addItemsAtribute_create_widgets(self):
      self.addItemsAtributeComplete = widgets.Label(value="- Items similarity computed -")
      self.addItemsAtributeTitle = widgets.HTML(value="<b>- Add atributes to compute items similarity -</b>")
      self.addItemsAtributeButton = widgets.Button(description="+ Atribute")
      self.addItemsAtributeButton.on_click(self.on_addItemsAtributeButton_clicked) 
      self.loadItemsAtributeButton = widgets.Button(description="Compute similarity")
      self.loadItemsAtributeButton.on_click(self.on_loadItemsAtributeButton_clicked)
    
    def addItemsAtribute(self):
      aux1 = widgets.Dropdown(options=self.items.columns,
                              value=None,
                              description='Atribute:',
                              disabled=False
                             )
      aux2 = widgets.Dropdown(options=self.items_sim_functions,
                              description='Sim Function::',
                              disabled=False
                             )
      aux3 = widgets.FloatText(value=0.0,
                               step=0.1,
                               description='Weight:',
                               disabled=False
                              )
      self.itemsLastAtribute = widgets.HBox([aux1, aux2, aux3])
    
    def on_addItemsAtributeButton_clicked(self, change):
      self.out.clear_output()
      self.addItemsAtribute()
      with self.out:
        display(self.loadDataComplete)
        display(self.addItemsAtributeTitle)
        for atr in self.atribute_list:
          display(atr)
        display(self.itemsLastAtribute)
        display(self.addItemsAtributeButton)
        display(self.loadItemsAtributeButton)
        self.atribute_list.append(self.itemsLastAtribute)
      
    def on_loadItemsAtributeButton_clicked(self, change):
      self.out.clear_output()
      with self.out: print('...')
      self.computeItemsSim()
      self.out.clear_output()      
      self.addUsersAtribute_create_widgets()
      self.atribute_list = []
      with self.out:
        display(self.loadDataComplete)
        display(self.addItemsAtributeComplete)
        display(self.addUsersAtributeTitle)
        display(self.addUsersAtributeButton)
    
    ### 3. SIMILITUD USUARIOS ###
    def computeItemsSim(self):
      funcs = {x : None for x in self.items.columns}
      weights = {x : None for x in self.items.columns}  

      for atr in self.atribute_list:
        funcs.update({atr.children[0].value : atr.children[1].value})
        weights.update({atr.children[0].value : atr.children[2].value})

      AM = ArtworksMatrix(self.items, function_dict=funcs, weight_dict=weights)
      self.itemsMatrix = AM.getSimilarityMatrix()
    
    def addUsersAtribute_create_widgets(self):
      self.addUsersAtributeComplete = widgets.Label(value="- Users similarity computed -")
      self.addUsersAtributeTitle = widgets.HTML(value="<b>- Add atributes to compute users similarity -</b>")
      self.addUsersAtributeButton = widgets.Button(description="+ Atribute")
      self.addUsersAtributeButton.on_click(self.on_addUsersAtributeButton_clicked) 
      self.loadUsersAtributeButton = widgets.Button(description="Compute similarity")
      self.loadUsersAtributeButton.on_click(self.on_loadUsersAtributeButton_clicked)
    
    def addUsersAtribute(self):
      aux1 = widgets.Dropdown(options=self.users.columns,
                              value=None,
                              description='Atribute:',
                              disabled=False
                             )
      aux2 = widgets.Dropdown(options=self.users_sim_functions,
                              description='Sim Function::',
                              disabled=False
                             )
      aux3 = widgets.FloatText(value=0.0,
                               step=0.1,
                               description='Weight:',
                               disabled=False
                              )
      self.usersLastAtribute = widgets.HBox([aux1, aux2, aux3])
    
    def on_addUsersAtributeButton_clicked(self, change):
      self.out.clear_output()
      self.addUsersAtribute()
      with self.out:
        display(self.loadDataComplete)
        display(self.addItemsAtributeComplete)
        display(self.addUsersAtributeTitle)
        for atr in self.atribute_list:
          display(atr)
        display(self.usersLastAtribute)
        display(self.addUsersAtributeButton)
        display(self.loadUsersAtributeButton)
        self.atribute_list.append(self.usersLastAtribute)
    
    def on_loadUsersAtributeButton_clicked(self, change):
      self.out.clear_output()
      # self.computeItemsSim()
      with self.out:
        display(self.loadDataComplete)
        display(self.addItemsAtributeComplete)        
        display(self.addUsersAtributeComplete)
    
    ### MOSTAR TODO ###
    def display_widgets(self):
        self.loadData_create_widgets()
        self.out = widgets.Output()  # this is the output widget in which the df is displayed
        
        display(self.out)
        with self.out:
          display(self.loadDataTitle)
          display(widgets.VBox(
                              [
                                  self.usersDropdown,
                                  self.itemsDropdown,
                                  self.loadDataButton,
                              ]
                          )
                 )

      
something = foo()
something.display_widgets()

Output()

In [3]:
def elegirDatos(path, files):
  usersDropdown = widgets.Dropdown(
      options=files,
      value='Prado_users_emotions_OnePolarity.csv',
      description='Users:',
      disabled=False,
  )
  itemsDropdown = widgets.Dropdown(
      options=files,
      value='Prado_artworks_wikidata.csv',
      description='Items:',
      disabled=False,
  )
  return usersDropdown, itemsDropdown

def button_cargarDatos(b, path, usersDropdown, itemsDropdown):
  return cargarDatos(path, usersDropdown, itemsDropdown)
  
def cargarDatos(path, usersDropdown, itemsDropdown):
  users = pd.read_csv(path + usersDropdown.value, index_col=0)
  artworks = pd.read_csv(path + itemsDropdown.value)
  users['positive'] = users['positive'].apply(eval)
  users['negative'] = users['negative'].apply(eval)
  users['mixed'] = users['mixed'].apply(eval)
  return users, artworks

def elegirSimilitudItems(path, files, artworks):
  functions = [None]
  functions.extend(inner_classes_list(SimilarityArtworks))

  dropdowns = {}
  for col in artworks.columns:
    dropdowns[col] = widgets.Dropdown(options=functions, value=None, description=col, disabled=False)

  colorDropdown = widgets.Dropdown(options=files, value=None, description='Colors', disabled=False)

  return dropdowns, colorDropdown

def elegirSimilitudUsers(path, files, users):
  functions = [None]
  functions.extend(inner_classes_list(SimilarityUsers))

  dropdowns = {}
  for col in users.columns:
    dropdowns[col] = widgets.Dropdown(options=functions, value=None, description=col, disabled=False)
  
  return dropdowns

def elegirPesosItems(atr_dropdowns, colorDropdown):
  floatTexts = {}
  colorFloatText = None
  for key, value in atr_dropdowns.items():
    # if value.value is not None:
      floatTexts[key] = widgets.FloatText(value=0.3, description=key, disabled=False)

  # if colorDropdown.value is not None:
      colorFloatText = widgets.FloatText(value=0.3, description='Colors', disabled=False)
    
  return floatTexts, colorFloatText

def elegirPesosUsers(atr_dropdowns):
  floatTexts = {}
  for key, value in atr_dropdowns.items():
    if value.value is not None:
      floatTexts[key] = widgets.FloatText(value=0.3, description=key, disabled=False)
  
  demographicFloatText = widgets.FloatText(value=0.5, description='Demographic', disabled=False)
  polarityFloatText = widgets.FloatText(value=0.5, description='Polarity', disabled=False)
  
  return floatTexts, demographicFloatText, polarityFloatText

def calcularSimilitudItems(path, artworks, atr_dropdowns, float_texts):
  funcs = {x : None for x in artworks.columns}
  weights = {x : None for x in artworks.columns}

  for key, value in atr_dropdowns.items():
    funcs.update({key: value.value})

  for key, value in float_texts.items():
    weights.update({key: value.value})
  weights.update({'Colors': colorFloatText.value})

  AM = ArtworksMatrix(artworks, function_dict=funcs, weight_dict=weights, colors_path=path + colorDropdown.value)
  
  return AM.getSimilarityMatrix()

def calcularSimilitudUsers(path, users, artworks_sim, atr_dropdowns, float_texts, demographicFloatText, polarityFloatText):
  funcs = {x : None for x in users.columns}
  weights = {x : None for x in users.columns}

  for key, value in atr_dropdowns.items():
    funcs.update({key: value.value})

  for key, value in float_texts.items():
    weights.update({key: value.value})

  weights.update({'polarity' : polarityFloatText.value, 'demographic' : demographicFloatText.value})  
  
  UM = UsersMatrix(users, artworks_sim, function_dict=funcs, weight_dict=weights)
  
  return UM.getSimilarityMatrix()

def calcularClusters(users, users_sim):
  users_distance = 1 - users_sim
  labels = UsersClustering(users_sim).kMedoidsFromMatrix()
  users_clustered = users.copy()
  users_clustered['cluster'] = labels
  return users_clustered

In [4]:
import asyncio

from IPython.display import display
import ipywidgets as widgets


out = widgets.Output(layout={'border': '1px solid black'})

def wait_for_click(btn, n_clicked=0):
    future = asyncio.Future()
    
    def on_button_clicked(btn):
        # btn.description = f"Clicked {n_clicked} times!"
        future.set_result(btn.description)
    
    btn.on_click(on_button_clicked)
    
    return future


n_changes = 1

btn = widgets.Button(description="Never Clicked")

async def f():
    for i in range(n_changes):
        data_path = 'data/'
        files = [f for f in listdir(data_path) if isfile(join(data_path, f))]
        
        usersDropdown, itemsDropdown = elegirDatos(data_path, files)
        display(itemsDropdown)
        display(usersDropdown)
        display(btn)

        btn_descr = await wait_for_click(btn, n_clicked=i + 1)
        
          # out.append_stdout(f'\nasync function continued with button description "{btn_descr}"\n')

        users, artworks = cargarDatos(data_path, usersDropdown, itemsDropdown)
        atr_dropdowns, colorDropdown = elegirSimilitudItems(data_path, files, artworks)
        for key, value in atr_dropdowns.items():
          display(value)
        display(usersDropdown)   
        out.append_stdout(f'\nasync function continued with button description "{btn_descr}"\n')
        
        btn = widgets.Button(description="Clicked Once")
        display(btn)
        
        btn_descr = await wait_for_click(btn, n_clicked=i + 1)
        out.append_stdout(f'\nasync function continued with button description "{btn_descr}"\n')

aux = asyncio.ensure_future(f())
out


Output(layout=Layout(border='1px solid black'))

In [5]:
out = widgets.Output(layout={'border': '1px solid black'})

In [6]:
@out.capture(clear_output=True)
def paso2(b):
  button = widgets.Button(
    description='+ Atribute',
    layout={'width': '300px'}
  )
  button.on_click(paso3)
  display(button)

def paso3(b):
  users, artworks = cargarDatos()
  dropdown = widgets.Dropdown(options=artworks, value=None, description=col, disabled=False)
  
  button = widgets.Button(
    description='+ Atribute',
    layout={'width': '300px'}
  )
  button.on_click(paso3)
  display(button)

In [7]:
out

Output(layout=Layout(border='1px solid black'))

Dropdown(description='Items:', index=8, options=('artworkColors.json', 'artworkEmotions.json', 'Clean_users.cs…

Dropdown(description='Users:', index=14, options=('artworkColors.json', 'artworkEmotions.json', 'Clean_users.c…

In [8]:
out.clear_output()
with out:
  data_path = 'data/'
  files = [f for f in listdir(data_path) if isfile(join(data_path, f))]

  usersDropdown, itemsDropdown = elegirDatos(data_path, files)
  display(itemsDropdown)
  display(usersDropdown)

  button = widgets.Button(
      description='DONE',
      layout={'width': '300px'}
  )

  button.on_click(functools.partial(button_cargarDatos, path=data_path, usersDropdown=usersDropdown, itemsDropdown=itemsDropdown))
  display(button)

# ----------------------------------------------

In [49]:
out.clear_output()

In [50]:
out = widgets.Output(layout={'border': '1px solid black'})
out

Output(layout=Layout(border='1px solid black'))

In [51]:
out.clear_output()
with out:
  data_path = 'data/'
  files = [f for f in listdir(data_path) if isfile(join(data_path, f))]

  usersDropdown, itemsDropdown = elegirDatos(data_path, files)
  display(itemsDropdown)
  display(usersDropdown)

In [55]:
out.clear_output()
with out:
  users, artworks = cargarDatos(data_path, usersDropdown, itemsDropdown)
  atr_dropdowns, colorDropdown = elegirSimilitudItems(data_path, files, artworks)
  
  float_texts, colorFloatText = elegirPesosItems(atr_dropdowns, colorDropdown)
  
  for key, value in atr_dropdowns.items():
    # display(value)
    display(widgets.HBox([value, float_texts.get(key)]))
    
  display(colorDropdown)

In [45]:
float_texts

{'ID': FloatText(value=0.3, description='ID'),
 'Category': FloatText(value=0.3, description='Category'),
 'Artist': FloatText(value=0.3, description='Artist'),
 'Title': FloatText(value=0.3, description='Title'),
 'Year': FloatText(value=0.3, description='Year'),
 'Image URL': FloatText(value=0.3, description='Image URL'),
 'Painting Info URL': FloatText(value=0.3, description='Painting Info URL'),
 'Artist Info URL': FloatText(value=0.3, description='Artist Info URL'),
 'Location': FloatText(value=0.3, description='Location'),
 'wd:paintingID': FloatText(value=0.3, description='wd:paintingID'),
 'wd:paintingURL': FloatText(value=0.3, description='wd:paintingURL'),
 'wd:artistID': FloatText(value=0.3, description='wd:artistID'),
 'wd:artistURL': FloatText(value=0.3, description='wd:artistURL')}

In [44]:
atributes_artworks = [key for key,value in atr_dropdowns.items() if value is not None]

float_texts, colorFloatText = elegirPesosItems(atr_dropdowns, colorDropdown)
for key, value in float_texts.items():
  display(value)

if colorDropdown.value is not None:
  display(colorFloatText) 

FloatText(value=0.3, description='ID')

FloatText(value=0.3, description='Category')

FloatText(value=0.3, description='Artist')

FloatText(value=0.3, description='Title')

FloatText(value=0.3, description='Year')

...

FloatText(value=0.3, description='Location')

FloatText(value=0.3, description='wd:paintingID')

FloatText(value=0.3, description='wd:paintingURL')

FloatText(value=0.3, description='wd:artistID')

FloatText(value=0.3, description='wd:artistURL')

In [33]:
widgets.HBox([itemsDropdown, usersDropdown, colorDropdown])